In [1]:
!pip install eli5

     |████████████████████████████████| 112kB 2.9MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance


In [5]:
ls

drive/  sample_data/


In [8]:
cd "drive/My Drive/Colab Notebooks"

/content/drive/My Drive/Colab Notebooks


In [9]:
ls

 challenge-matrix/       'Copy of cifar10.ipynb'   Untitled1.ipynb
'challenge-matrix (1)'/   day4.ipynb               Untitled2.ipynb
 cifar10.ipynb            day4_ML.ipynb
 CoLabDriveGit.ipynb      Untitled0.ipynb


In [10]:
cd "challenge-matrix"

/content/drive/My Drive/Colab Notebooks/challenge-matrix


In [11]:
cd data

/content/drive/My Drive/Colab Notebooks/challenge-matrix/data


In [12]:
ls

shoes_prices2.csv  shoes_prices.csv


In [0]:
df = pd.read_csv('shoes_prices2.csv', low_memory=False)

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df [feats].values
  y = df ['prices_amountmin'].values
  #model = DecisionTreeRegressor
  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [0]:
df['brand_cat'] = df.brand.map(lambda x: str(x).lower()).factorize()[0]

In [0]:
feats = ['brand_cat']

In [23]:
run_model(feats)

(-58.133398968282776, 4.206122611474276)

In [24]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=1)
run_model(feats, model)

(-57.285351528077264, 4.131927834435122)

In [0]:
from ast import literal_eval
from tqdm import tqdm_notebook

In [0]:
def parse_features(x):
  output_dict = {}
  if (str(x) == 'nan'): return output_dict
  features = literal_eval(x.replace('\\"','"'))

  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key] = value
  return output_dict


df['features_parsed'] = df ['features'].map(parse_features)

In [30]:
df['features_parsed'].head()

0    {'gender': 'men', 'shoe size': 'm', 'shoe cate...
1    {'gender': 'men', 'shoe size': 'm', 'shoe cate...
2    {'gender': 'men', 'color': 'black', 'shipping ...
3    {'gender': 'men', 'color': 'black', 'shipping ...
4    {'gender': 'men', 'color': 'black', 'shipping ...
Name: features_parsed, dtype: object

In [31]:
keys = set()

df['features_parsed'].map(lambda x: keys.update(x.keys()))

0        None
1        None
2        None
3        None
4        None
         ... 
18275    None
18276    None
18277    None
18278    None
18279    None
Name: features_parsed, Length: 18280, dtype: object

In [32]:
len(keys)

476

In [35]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df [get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [36]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_special features', 'feat_band color', 'feat_neckline',
       'feat_color mapping', 'feat_eye size:', 'feat_use', 'feat_sport',
       'feat_flame resistant', 'feat_chest', 'feat_productgroup'],
      dtype='object', length=526)

In [0]:
keys_stat = {}
for key in keys:
  keys_stat[key] = df [False == df[get_name_feat(key)].isnull()].shape[0] / df.shape[0] * 100

In [47]:
{k: v for k,v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df.feat_brand.map(lambda x: str(x).lower()).factorize()[0]

In [0]:
df['feat_color_cat'] = df.feat_color.map(lambda x: str(x).lower()).factorize()[0]

In [0]:
df['feat_gender_cat'] = df.feat_gender.map(lambda x: str(x).lower()).factorize()[0]

In [0]:
#df['feat_manufacturer_part_number'] = df['manufacturer_part_number'].map(lambda x: str(x).lower()).factorize()[0]

In [0]:
df['feat_material_cat'] = df.feat_material.map(lambda x: str(x).lower()).factorize()[0]

In [0]:
df['feat_sport_cat'] = df.feat_sport.map(lambda x: str(x).lower()).factorize()[0]

In [0]:
df['feat_style_cat'] = df.feat_style.map(lambda x: str(x).lower()).factorize()[0]

In [0]:
for key in keys:
  df [get_name_feat(key) + "_cat"] = df[get_name_feat(key)].map(lambda x: str(x).lower()).factorize()[0]

In [53]:
df [df.brand != df.feat_brand].shape

(18228, 530)

In [56]:
df [df.brand == df.feat_brand].shape


(8846, 530)

In [0]:
df['brand'] = df ['brand'].map(lambda x: str(x).lower())

In [55]:
df [df.brand != df.feat_brand].shape

(9434, 530)

In [0]:
feats = ['']

In [0]:
feats_cat = [x for x in df.columns if 'cat' in x]

In [77]:
feats_cat

['categories',
 'brand_cat',
 'feat_recommended location',
 'feat_clothing category',
 'feat_certifications and listings',
 'feat_catalog',
 'feat_location - country',
 'feat_shoe category',
 'feat_multi pack indicator',
 'feat_location - city/state',
 'feat_fabrication',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'feat_garment care_cat',
 'feat_golf shoe type_cat',
 'feat_main color_cat',
 'feat_contains cubic zirconia_cat',
 'feat_seller inventory number_cat',
 'feat_number of compartments_cat',
 'feat_face shape_cat',
 'feat_boxed-product dimensions_cat',
 'feat_removable liner_cat',
 'feat_gold karat_cat',
 'feat_upper material_cat',
 'feat_primary shelf id_cat',
 'feat_bed size_cat',
 'feat_color/finish family_cat',
 'feat_has adaptive lenses_cat',
 'feat_shipping weight (in pounds)_cat',
 'feat_country of manufacturer_cat',
 'feat_subdial_cat',
 'feat_item style_cat',
 'feat_lens color family_cat',
 'feat

In [59]:
model = RandomForestRegressor (max_depth= 5, n_estimators=100)
run_model(['brand_cat'],model)

(-57.30341352535733, 4.167165636631581)

In [80]:
model = RandomForestRegressor (max_depth= 5, n_estimators=100)
feats = ['brand_cat', 'feat_style_cat', 'feat_sport_cat','feat_brand_cat', 'feat_gender_cat',"feat_material_cat"]
#feats += feats_cat
#feats = list(set(feats))
feats += ['feat_label_cat',
 'feat_pattern or design_cat',
 'feat_authenticity_cat',
 'feat_style name_cat',
 'feat_diamond clarity_cat',
 'feat_colour code_cat',
 'feat_sku number_cat',
 'feat_fits model_cat',
 'feat_is recyclable_cat',
 'feat_szie_cat',
 'feat_lining_cat',
 'feat_closure style_cat']
run_model(feats,model)

(-57.150346389076574, 4.3669848728475325)

In [81]:
m = RandomForestRegressor(max_depth= 5, n_estimators=100, random_state=1)
X = df [feats].values
y = df['prices_amountmin']
m.fit(X, y)

perm = PermutationImportance(m, random_state=1).fit(X, y);
eli5.show_weights(perm, feature_names=feats)

Weight,Feature
0.2647 ± 0.0115,brand_cat
0.1133 ± 0.0099,feat_material_cat
0.0430 ± 0.0016,feat_gender_cat
0.0218 ± 0.0014,feat_brand_cat
0.0074 ± 0.0005,feat_style_cat
0.0019 ± 0.0003,feat_diamond clarity_cat
0.0001 ± 0.0000,feat_sport_cat
0.0000 ± 0.0000,feat_authenticity_cat
0 ± 0.0000,feat_style name_cat
0 ± 0.0000,feat_lining_cat


In [68]:
df.brand.value_counts(normalize=True)

nike                       0.097210
puma                       0.033315
ralph lauren               0.028775
vans                       0.021116
new balance                0.020295
                             ...   
croft & barrow (kohl's)    0.000055
fuel                       0.000055
american eagle             0.000055
sod                        0.000055
wisconsin badgers          0.000055
Name: brand, Length: 1732, dtype: float64

In [69]:
df [df['brand'] == 'nike'].features_parsed.head().values

array([{'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'style': 'athletic sneakers', 'condition': 'new with box'}, {}],
      dtype=object)

In [89]:
ls

 challenge-matrix/       'Copy of cifar10.ipynb'   Untitled0.ipynb
'challenge-matrix (1)'/   day4.ipynb               Untitled1.ipynb
 cifar10.ipynb            day4_ML.ipynb
 CoLabDriveGit.ipynb      day5.ipynb


In [0]:
cd "challeng"